In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! pip install -q langchain langchain-community pypdf
! pip install -q "langchain>=0.3.0" "langchain-community>=0.3.0" chromadb "sentence-transformers>=3.0.0"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 96.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.5/329.5 kB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 67.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.4/21.4 MB 64.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from pathlib import Path

from pathlib import Path

import torch
import glob
import re

In [ ]:
PDF_DIR = '/content/drive/Othercomputers/My Laptop/Docs/multimodal-rag/data/raw/text'

In [ ]:
def load_all_pdfs(pdf_dir: str, skip_names: list[str] | None = None):
    if skip_names is None:
        skip_names = ['QUY CHUẨN KỸ THUẬT QUỐC GIA VỀ BÁO HIỆU ĐƯỜNG BỘ',
                      'cấu tạo và sửa chữa']

    pdf_paths = glob.glob(str(Path(pdf_dir) / "*.pdf"))
    docs = []

    for path in pdf_paths:
        name = Path(path).name
        # Bỏ qua những file nặng / không muốn load
        if any(skip in name for skip in skip_names):
            print(f"[SKIP] {name}")
            continue

        print(f"[LOAD] {name}")
        loader = PyPDFLoader(path)
        pdf_docs = loader.load()  # Mỗi page là 1 Document
        for d in pdf_docs:
            d.metadata["source_file"] = name
        docs.extend(pdf_docs)

    return docs


In [ ]:
law_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=150,
    length_function=len,
    is_separator_regex=True,
    separators=[
        # CHƯƠNG I, Chương I, chương I, CHƯƠNG I. QUY ĐỊNH CHUNG...
        r"(?i)\n{2,}chương\s+[ivxlcdm]+\s.*",
        # MỤC 1, Mục 1. Mục 1: ...
        r"(?i)\n{2,}mục\s+\d+\.?.*",
        # ĐIỀU 1, Điều 1. ĐIỀU 1: ...
        r"(?i)\n{2,}điều\s+\d+\.?.*",
        # Các khoản trong điều: 1. 2. 3. ... (đầu dòng, có thể 1 hoặc nhiều newline)
        r"\n{1,}\d+\.\s",
        # Tiểu mục nếu bạn muốn bắt luôn (TIỂU MỤC 1, Tiểu mục 2...)
        # r"(?i)\n{2,}tiểu\s+mục\s+\d+\.?.*",

        # Đoạn
        r"\n{2,}",
        # Dòng
        r"\n",
        # Khoảng trắng
        r" ",
        # Fallback
        r"",
    ],
)


In [ ]:
# merged_docs = load_pdfs_merged(PDF_DIR)
raw_docs = load_all_pdfs(PDF_DIR)
print(raw_docs)

chunked_docs = law_splitter.split_documents(raw_docs)

print(len(chunked_docs))
print(type(chunked_docs))

[LOAD] Luật đường bộ.pdf
[LOAD] LUẬT TRẬT TỰ, AN TOÀN GIAO THÔNG ĐƯỜNG BỘ.pdf
[SKIP] QUY CHUẨN KỸ THUẬT QUỐC GIA VỀ BÁO HIỆU ĐƯỜNG BỘ.pdf
[LOAD] Quy định về tốc độ và khoảng cách an toàn của xe cơ giới, xe máy chuyên dùng tham gia giao thông trên đường bộ.pdf
[LOAD] Luật phòng cháy chữa cháy.pdf
[SKIP] cấu tạo và sửa chữa.pdf
[LOAD] Cứ nạn tai nạn giao thông.pdf
[LOAD] Mẹo ghi nhớ.pdf
[LOAD] Luật phòng chống tác hại rượu bia.pdf
[Document(metadata={'producer': 'PyPDF', 'creator': 'PyPDF', 'creationdate': '', 'source': '/content/drive/Othercomputers/My Laptop/Docs/multimodal-rag/data/raw/text/Luật đường bộ.pdf', 'total_pages': 69, 'page': 0, 'page_label': '1', 'source_file': 'Luật đường bộ.pdf'}, page_content='CÔNG BÁO/Số 983 + 984/Ngày 25-8-2024 3 \n \n \nQUỐC HỘI \n \nCỘNG HÒA XÃ HỘI CHỦ NGHĨA VIỆT NAM \nĐộc lập - Tự do - Hạnh phúc \nLuật số: 35/2024/QH15  \n \nLUẬT \nĐƯỜNG BỘ 

In [ ]:
OUT_DIR = Path("/content/drive/Othercomputers/My Laptop/Docs/multimodal-rag/data/outputs")
OUT_DIR.mkdir(parents=True, exist_ok=True)

In [ ]:
txt_path = OUT_DIR / "chunked_docs.txt"

with open(txt_path, "w", encoding="utf-8") as f:
    for i, doc in enumerate(chunked_docs):
        f.write(f"===== CHUNK {i} =====\n")
        f.write(f"Source: {doc.metadata.get('source_file', '')}\n")
        f.write(f"Page: {doc.metadata.get('page', '')}\n\n")
        f.write(doc.page_content)
        f.write("\n\n\n")

print("Đã lưu file TXT tại:", txt_path)


Đã lưu file TXT tại: /content/drive/Othercomputers/My Laptop/Docs/multimodal-rag/data/outputs/chunked_docs.txt


In [ ]:
EMBED_MODEL_NAME = "AITeamVN/Vietnamese_Embedding"

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)

embeddings = HuggingFaceEmbeddings(
    model_name=EMBED_MODEL_NAME,
    model_kwargs={"device": device},
    encode_kwargs={"normalize_embeddings": True},  # thường tốt cho search cosine
)

Device: cuda


/tmp/ipython-input-3032504815.py:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/171 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

In [ ]:
CHROMA_DIR = Path("/content/drive/Othercomputers/My Laptop/Docs/multimodal-rag/chroma_text_law")
CHROMA_DIR.mkdir(parents=True, exist_ok=True)

vectorstore = Chroma.from_documents(
    documents=chunked_docs,       # chính là list Document của bạn
    embedding=embeddings,
    persist_directory=str(CHROMA_DIR),
)

vectorstore.persist()
print("Đã build xong Chroma, lưu tại:", CHROMA_DIR)

Đã build xong Chroma, lưu tại: /content/drive/Othercomputers/My Laptop/Docs/multimodal-rag/chroma_text_law


/tmp/ipython-input-2894211323.py:10: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


In [ ]:
retriever = vectorstore.as_retriever(
    search_type="similarity",      # hoặc "mmr"
    search_kwargs={"k": 5},        # lấy top 5 chunk
)

In [ ]:
# Ví dụ truy vấn
query = "Quy định về tốc độ tối đa cho ô tô trên đường cao tốc là gì?"

docs = retriever.invoke(query)  # LangChain >= 0.3 dùng .invoke()

print(f"Tìm được {len(docs)} chunk\n")
for i, d in enumerate(docs, start=1):
    print(f"===== KẾT QUẢ {i} =====")
    print("Source:", d.metadata.get("source_file"))
    print("Page:", d.metadata.get("page"))
    print("Nội dung:")
    print(d.page_content[:500], "...")
    print()

Tìm được 5 chunk

===== KẾT QUẢ 1 =====
Source: Quy định về tốc độ và khoảng cách an toàn của xe cơ giới, xe máy chuyên dùng tham gia giao thông trên đường bộ.pdf
Page: 4
Nội dung:
5
Điều 9. Tốc độ khai thác tối đa, tốc độ khai thác tối thiểu cho phép đối 
với các loại xe cơ giới, xe máy chuyên dùng trên đường cao tốc
1.Đường cao tốc phải được đặt biển báo tốc độ khai thác tối đa, tốc độ khai 
thác tối thiểu.
2.Tốc độ khai thác tối đa cho phép trên đường cao tốc là 120 km/h.
3.Tốc độ khai thác tối thiểu cho phép trên đường cao tốc là 60 km/h. Trường 
hợp đường cao tốc có tốc độ thiết kế 60 km/h thì tốc độ khai thác tối thiểu thực 
hiện theo phương án tổ chức giao thông đượ ...

===== KẾT QUẢ 2 =====
Source: Quy định về tốc độ và khoảng cách an toàn của xe cơ giới, xe máy chuyên dùng tham gia giao thông trên đường bộ.pdf
Page: 4
Nội dung:
5
Điều 9. Tốc độ khai thác tối đa, tốc độ khai thác tối thiểu cho phép đối 
với các loại xe cơ giới,